In [1]:
import os
import os.path as osp
import cv2
import torch
import torchvision
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from torchsummary import summary
from PIL import Image
import numpy as np
i2t = transforms.ToTensor()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Torch device:', device)

Torch device: cuda


In [2]:
class ResNet18(nn.Module):
    def __init__(self, num_classes, **kwargs):
        super(ResNet18, self).__init__()
        resnet = torchvision.models.resnet18(pretrained=True)
        self.base = nn.Sequential(*list(resnet.children())[:-2])
        self.classifier = nn.Linear(resnet.fc.in_features, num_classes)
        self.feat_dim = resnet.fc.in_features # feature dimension

    def forward(self, x):
        x = self.base(x)
        x = F.avg_pool2d(x, x.size()[2:])
        f = x.view(x.size(0), -1)
        y = self.classifier(f)
        return y

In [3]:
def load_resnet(model_path):
    resnet = ResNet18(num_classes=16)
    resnet.load_state_dict(torch.load(model_path))
    modules = list(resnet.children())[:-1]
    resnet = nn.Sequential(*modules).to(device)
    for param in resnet.parameters():
        param.requires_grad = False
    return resnet

In [4]:
def extract_frames(video_path, skip_frame=8):
    vidcap = cv2.VideoCapture(video_path)
    success,image = vidcap.read()
    count = 0
    frames = []
    while success:
      if count%8 == 0:
        frames.append(image)
      success,image = vidcap.read()      
      count += 1
    return frames

In [5]:
def extract_feats(resnet, video_frames, device):
    feats = np.ndarray((len(video_frames), 7, 7, 512))
    for idx, frame in enumerate(video_frames):
        frame_img = Image.fromarray(frame).resize((224, 224), Image.ANTIALIAS)
        img_tensor = i2t(frame_img).view(1, 3, 224, 224).to(device)
        output = resnet(img_tensor).transpose(1,2).transpose(2,3)
        feats[idx, :, :, :] = output.cpu()
    return feats

In [6]:
def safe_mkdirs(path):
    if not osp.exists(path):
        os.makedirs(path)

In [7]:
model_path = '/home/zal/Devel/Vehice_Action_Classifier/src/pytorch/models/resnet18_pretrained_bs_128_aug_rot_acc_70v0.pt'
resnet = load_resnet(model_path)

In [12]:
source_dir = '/media/zal/Alfheim/Data/VIRAT/Proposals/'
target_dir = '/media/zal/Alfheim/Data/VIRAT/OrientationFeats/'

In [13]:
for proposal in ['gt', 'heu_gt_obj_props', 'hty_v1_props']:
    safe_mkdirs(osp.join(target_dir, proposal))
    for videoclip_dir in os.listdir(osp.join(source_dir, proposal)):
        print('Extracting feats of', videoclip_dir)
        video_path = osp.join(source_dir, proposal, videoclip_dir, 'video.mp4')
        frames = extract_frames(video_path, 8)
        feats = extract_feats(resnet, frames, device)
        feats_save_path = osp.join(target_dir, proposal, '%s.npz'%videoclip_dir)
        np.savez_compressed(feats_save_path, feat=feats)

Extracting feats of VIRAT_S_000000_50001
Extracting feats of VIRAT_S_000000_50002
Extracting feats of VIRAT_S_000000_50003
Extracting feats of VIRAT_S_000000_50004
Extracting feats of VIRAT_S_000000_50005
Extracting feats of VIRAT_S_000000_50006
Extracting feats of VIRAT_S_000000_50007
Extracting feats of VIRAT_S_000000_50008
Extracting feats of VIRAT_S_000000_50009
Extracting feats of VIRAT_S_000000_50010
Extracting feats of VIRAT_S_000000_50011
Extracting feats of VIRAT_S_000000_50012
Extracting feats of VIRAT_S_000000_50013
Extracting feats of VIRAT_S_000000_50014
Extracting feats of VIRAT_S_000000_50015
Extracting feats of VIRAT_S_000000_50016
Extracting feats of VIRAT_S_000000_50017
Extracting feats of VIRAT_S_000000_50018
Extracting feats of VIRAT_S_000000_50019
Extracting feats of VIRAT_S_000000_50020
Extracting feats of VIRAT_S_000000_50021
Extracting feats of VIRAT_S_000000_50022
Extracting feats of VIRAT_S_000204_05_001334_001520_50019
Extracting feats of VIRAT_S_000204_05_00

Extracting feats of VIRAT_S_040000_08_001084_001190_50011
Extracting feats of VIRAT_S_040000_08_001084_001190_50012
Extracting feats of VIRAT_S_040000_09_001194_001574_50001
Extracting feats of VIRAT_S_040000_09_001194_001574_50002
Extracting feats of VIRAT_S_040000_09_001194_001574_50003
Extracting feats of VIRAT_S_040000_09_001194_001574_50004
Extracting feats of VIRAT_S_040000_09_001194_001574_50005
Extracting feats of VIRAT_S_040000_09_001194_001574_50006
Extracting feats of VIRAT_S_040000_09_001194_001574_50007
Extracting feats of VIRAT_S_040000_09_001194_001574_50008
Extracting feats of VIRAT_S_040000_09_001194_001574_50009
Extracting feats of VIRAT_S_040104_06_001121_001241_50006
Extracting feats of VIRAT_S_040104_06_001121_001241_50007
Extracting feats of VIRAT_S_040104_06_001121_001241_50008
Extracting feats of VIRAT_S_040104_06_001121_001241_50009
Extracting feats of VIRAT_S_040104_06_001121_001241_50010
Extracting feats of VIRAT_S_040104_06_001121_001241_50011
Extracting fea

Extracting feats of VIRAT_S_000203_03_000736_000785_50013
Extracting feats of VIRAT_S_000203_05_001122_001159_50001
Extracting feats of VIRAT_S_000203_05_001122_001159_50002
Extracting feats of VIRAT_S_000203_05_001122_001159_50003
Extracting feats of VIRAT_S_000203_06_001206_001266_50001
Extracting feats of VIRAT_S_000203_06_001206_001266_50002
Extracting feats of VIRAT_S_000203_06_001206_001266_50003
Extracting feats of VIRAT_S_040001_01_000448_001101_50006
Extracting feats of VIRAT_S_040001_01_000448_001101_50009
Extracting feats of VIRAT_S_040001_01_000448_001101_50010
Extracting feats of VIRAT_S_040001_01_000448_001101_50013
Extracting feats of VIRAT_S_040001_01_000448_001101_50015
Extracting feats of VIRAT_S_040001_01_000448_001101_50016
Extracting feats of VIRAT_S_040001_01_000448_001101_50018
Extracting feats of VIRAT_S_040001_01_000448_001101_50019
Extracting feats of VIRAT_S_040001_01_000448_001101_50020
Extracting feats of VIRAT_S_040001_01_000448_001101_50021
Extracting fea

Extracting feats of VIRAT_S_000204_04_000738_000977_50042
Extracting feats of VIRAT_S_000204_04_000738_000977_50043
Extracting feats of VIRAT_S_000204_04_000738_000977_50044
Extracting feats of VIRAT_S_000204_04_000738_000977_50045
Extracting feats of VIRAT_S_000204_04_000738_000977_50046
Extracting feats of VIRAT_S_000204_04_000738_000977_50047
Extracting feats of VIRAT_S_000204_04_000738_000977_50048
Extracting feats of VIRAT_S_000204_04_000738_000977_50049
Extracting feats of VIRAT_S_000204_04_000738_000977_50050
Extracting feats of VIRAT_S_000204_04_000738_000977_50051
Extracting feats of VIRAT_S_000204_04_000738_000977_50052
Extracting feats of VIRAT_S_000204_04_000738_000977_50053
Extracting feats of VIRAT_S_000204_04_000738_000977_50054
Extracting feats of VIRAT_S_000204_04_000738_000977_50055
Extracting feats of VIRAT_S_000204_04_000738_000977_50056
Extracting feats of VIRAT_S_000204_04_000738_000977_50057
Extracting feats of VIRAT_S_000204_04_000738_000977_50058
Extracting fea

Extracting feats of VIRAT_S_040003_04_000758_001118_50036
Extracting feats of VIRAT_S_040003_04_000758_001118_50037
Extracting feats of VIRAT_S_040003_04_000758_001118_50038
Extracting feats of VIRAT_S_040003_04_000758_001118_50039
Extracting feats of VIRAT_S_040003_04_000758_001118_50040
Extracting feats of VIRAT_S_040003_04_000758_001118_50041
Extracting feats of VIRAT_S_040003_04_000758_001118_50042
Extracting feats of VIRAT_S_040003_04_000758_001118_50043
Extracting feats of VIRAT_S_040003_04_000758_001118_50044
Extracting feats of VIRAT_S_040003_04_000758_001118_50045
Extracting feats of VIRAT_S_040003_04_000758_001118_50046
Extracting feats of VIRAT_S_040003_04_000758_001118_50047
Extracting feats of VIRAT_S_040003_04_000758_001118_50048
Extracting feats of VIRAT_S_040003_04_000758_001118_50049
Extracting feats of VIRAT_S_040003_04_000758_001118_50050
Extracting feats of VIRAT_S_040003_04_000758_001118_50051
Extracting feats of VIRAT_S_040102_08_000986_001090_50024
Extracting fea

...

Extracting feats of VIRAT_S_040103_01_000132_000195_648
Extracting feats of VIRAT_S_040103_01_000132_000195_649
Extracting feats of VIRAT_S_040103_01_000132_000195_650
Extracting feats of VIRAT_S_050000_09_001310_001373_58
Extracting feats of VIRAT_S_050000_09_001310_001373_59
Extracting feats of VIRAT_S_050000_09_001310_001373_6
Extracting feats of VIRAT_S_050000_09_001310_001373_60
Extracting feats of VIRAT_S_050000_09_001310_001373_61
Extracting feats of VIRAT_S_050000_09_001310_001373_62
Extracting feats of VIRAT_S_050000_09_001310_001373_63
Extracting feats of VIRAT_S_050000_09_001310_001373_64
Extracting feats of VIRAT_S_050000_09_001310_001373_65
Extracting feats of VIRAT_S_050000_09_001310_001373_66
Extracting feats of VIRAT_S_050000_09_001310_001373_67
Extracting feats of VIRAT_S_050000_09_001310_001373_68
Extracting feats of VIRAT_S_050000_09_001310_001373_69
Extracting feats of VIRAT_S_050000_09_001310_001373_7
Extracting feats of VIRAT_S_050000_09_001310_001373_70
Extractin

Extracting feats of VIRAT_S_000203_03_000736_000785_42
Extracting feats of VIRAT_S_000203_03_000736_000785_43
Extracting feats of VIRAT_S_000203_03_000736_000785_44
Extracting feats of VIRAT_S_000203_03_000736_000785_45
Extracting feats of VIRAT_S_000203_03_000736_000785_46
Extracting feats of VIRAT_S_000201_01_000384_000589_4078
Extracting feats of VIRAT_S_000201_01_000384_000589_4079
Extracting feats of VIRAT_S_000201_01_000384_000589_4080
Extracting feats of VIRAT_S_000201_01_000384_000589_4081
Extracting feats of VIRAT_S_000201_01_000384_000589_4082
Extracting feats of VIRAT_S_000201_01_000384_000589_4083
Extracting feats of VIRAT_S_000201_01_000384_000589_4084
Extracting feats of VIRAT_S_000201_01_000384_000589_4085
Extracting feats of VIRAT_S_000201_01_000384_000589_4086
Extracting feats of VIRAT_S_000201_01_000384_000589_4087
Extracting feats of VIRAT_S_000201_01_000384_000589_4088
Extracting feats of VIRAT_S_000201_01_000384_000589_4089
Extracting feats of VIRAT_S_000201_01_000

Extracting feats of VIRAT_S_000201_03_000640_000672_601
Extracting feats of VIRAT_S_000201_04_000682_000822_15
Extracting feats of VIRAT_S_000201_04_000682_000822_2299
Extracting feats of VIRAT_S_000201_04_000682_000822_27
Extracting feats of VIRAT_S_000201_04_000682_000822_4
Extracting feats of VIRAT_S_000201_05_001081_001215_22
Extracting feats of VIRAT_S_000201_05_001081_001215_2326
Extracting feats of VIRAT_S_050000_09_001310_001373_1065
Extracting feats of VIRAT_S_050000_09_001310_001373_1066
Extracting feats of VIRAT_S_050000_09_001310_001373_1067
Extracting feats of VIRAT_S_050000_09_001310_001373_1068
Extracting feats of VIRAT_S_050000_09_001310_001373_1069
Extracting feats of VIRAT_S_050000_09_001310_001373_1071
Extracting feats of VIRAT_S_050000_09_001310_001373_1072
Extracting feats of VIRAT_S_050000_09_001310_001373_1073
Extracting feats of VIRAT_S_050000_09_001310_001373_1075
Extracting feats of VIRAT_S_050000_09_001310_001373_11
Extracting feats of VIRAT_S_050000_09_00131

Extracting feats of VIRAT_S_000203_03_000736_000785_1510
Extracting feats of VIRAT_S_000203_03_000736_000785_16
Extracting feats of VIRAT_S_000203_03_000736_000785_17
Extracting feats of VIRAT_S_000203_03_000736_000785_18
Extracting feats of VIRAT_S_000205_02_000409_000566_4
Extracting feats of VIRAT_S_000205_02_000409_000566_40
Extracting feats of VIRAT_S_000205_02_000409_000566_41
Extracting feats of VIRAT_S_000205_02_000409_000566_42
Extracting feats of VIRAT_S_000205_02_000409_000566_43
Extracting feats of VIRAT_S_000205_02_000409_000566_44
Extracting feats of VIRAT_S_000205_02_000409_000566_45
Extracting feats of VIRAT_S_000205_02_000409_000566_46
Extracting feats of VIRAT_S_000205_02_000409_000566_47
Extracting feats of VIRAT_S_000205_02_000409_000566_48
Extracting feats of VIRAT_S_000205_02_000409_000566_49
Extracting feats of VIRAT_S_000205_02_000409_000566_5
Extracting feats of VIRAT_S_000205_02_000409_000566_50
Extracting feats of VIRAT_S_050000_06_000908_000970_13
Extracting

Extracting feats of VIRAT_S_040103_02_000199_000279_745
Extracting feats of VIRAT_S_040103_03_000284_000425_1414
Extracting feats of VIRAT_S_040103_03_000284_000425_1433
Extracting feats of VIRAT_S_040103_03_000284_000425_2
Extracting feats of VIRAT_S_040103_03_000284_000425_39
Extracting feats of VIRAT_S_040103_03_000284_000425_57
Extracting feats of VIRAT_S_040103_05_000729_000804_16
Extracting feats of VIRAT_S_000207_02_000498_000530_16
Extracting feats of VIRAT_S_000207_02_000498_000530_17
Extracting feats of VIRAT_S_000207_02_000498_000530_18
Extracting feats of VIRAT_S_000207_02_000498_000530_19
Extracting feats of VIRAT_S_000207_02_000498_000530_2
Extracting feats of VIRAT_S_000207_02_000498_000530_20
Extracting feats of VIRAT_S_000207_02_000498_000530_21
Extracting feats of VIRAT_S_000207_02_000498_000530_3
Extracting feats of VIRAT_S_000207_02_000498_000530_350
Extracting feats of VIRAT_S_000207_02_000498_000530_351
Extracting feats of VIRAT_S_000207_02_000498_000530_352
Extra